In [10]:
import logging
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import sklearn

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/Air-BnB-2-BW/data-science/master/airbnb_BW2.csv')

In [12]:
dataset = df.values

In [13]:
X = dataset[:,0:10]
y = dataset[:,10]
y = np.reshape(y, (-1,1))
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

In [14]:
scaler_x.fit(X)
xscale=scaler_x.transform(X)
scaler_y.fit(y)
yscale=scaler_y.transform(y)

In [15]:
X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y.reshape(len(y),1))[:,0]
X_train, X_test, y_train, y_test = train_test_split(xscale, yscale)

In [16]:
model = Sequential()
model.add(Dense(12, input_dim=10, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=50, batch_size=50,  verbose=1, validation_split=0.2)

Epoch 1/50
1014/1014 [==============================] - 1s 1ms/step - loss: 0.2212 - mse: 0.2212 - mae: 0.4383 - val_loss: 0.2153 - val_mse: 0.2153 - val_mae: 0.4317
Epoch 2/50
1014/1014 [==============================] - 1s 1ms/step - loss: 0.2133 - mse: 0.2133 - mae: 0.4278 - val_loss: 0.2157 - val_mse: 0.2157 - val_mae: 0.4187
Epoch 3/50
1014/1014 [==============================] - 1s 853us/step - loss: 0.2127 - mse: 0.2127 - mae: 0.4255 - val_loss: 0.2139 - val_mse: 0.2139 - val_mae: 0.4327
Epoch 4/50
1014/1014 [==============================] - 1s 1ms/step - loss: 0.2122 - mse: 0.2122 - mae: 0.4244 - val_loss: 0.2137 - val_mse: 0.2137 - val_mae: 0.4247
Epoch 5/50
1014/1014 [==============================] - 1s 1ms/step - loss: 0.2121 - mse: 0.2121 - mae: 0.4243 - val_loss: 0.2152 - val_mse: 0.2152 - val_mae: 0.4343
Epoch 6/50
1014/1014 [==============================] - 1s 911us/step - loss: 0.2120 - mse: 0.2120 - mae: 0.4238 - val_loss: 0.2148 - val_mse: 0.2148 - val_mae: 0.4166


In [17]:
model.save('keras_nn_model.h5')